In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install splink

In [ ]:
from splink import splink_datasets, DuckDBAPI, block_on
import splink.exploratory as exploratory

df = splink_datasets.historical_50k
df = df.rename(columns={"cluster": "cluster_ground_truth"})
df.head(5)


## Exploratory analysis

In [ ]:
import splink.exploratory as exploratory

db_api = DuckDBAPI()
exploratory.completeness_chart(df, db_api=db_api)

In [ ]:
db_api = DuckDBAPI()
exploratory.profile_columns(df, column_expressions=["first_name", "dob"], db_api=db_api)

## Blocking analysis

In [ ]:
from splink import blocking_analysis as ba
from splink import block_on

blocking_rule_for_analysis = block_on("first_name")

res = ba.count_comparisons_from_blocking_rule(
    table_or_tables=df,
    blocking_rule=blocking_rule_for_analysis,
    link_type="dedupe_only",
    db_api=db_api,
)
res

In [ ]:
num_comparisons = res["number_of_comparisons_to_be_scored_post_filter_conditions"]
print(
    f"Number of comparisons generated: {num_comparisons:,}"
)

In [ ]:
blocking_rule_for_analysis = block_on("substr(first_name, 1, 1)")

res = ba.count_comparisons_from_blocking_rule(
    table_or_tables=df,
    blocking_rule=blocking_rule_for_analysis,
    link_type="dedupe_only",
    db_api=db_api,
)

num_comparisons = res["number_of_comparisons_to_be_scored_post_filter_conditions"]
print(
    f"Number of comparisons generated: {num_comparisons:,}"
)

In [ ]:
blocking_rules = [
    block_on("first_name", "surname"),
    block_on("first_name", "dob"),
    block_on("dob", "postcode_fake"),
    block_on("occupation", "dob"),
    block_on("birth_place", "dob"),
    block_on("birth_place", "substr(postcode_fake,1,4)"),
    block_on("substr(first_name,1,2)", "surname"),

]

ba.cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    link_type="dedupe_only",
    blocking_rules=blocking_rules,
    table_or_tables=df,
    db_api=db_api,
)



In [ ]:
blocking_rule_for_analysis = block_on("first_name", "surname")

ba.n_largest_blocks(
    link_type="dedupe_only",
    blocking_rule=blocking_rule_for_analysis,
    table_or_tables=df,
    db_api=db_api,
).as_duckdbpyrelation()

## Define and estimate a model

```
Data Linking Model
├─-- Comparison: Date of birth
│    ├─-- ComparisonLevel: Exact match on date of birth with term frequency adjustments
│    ├─-- ComparisonLevel: DamerauLevenshtein distance <= 1
│    ├─-- ComparisonLevel: Abs date difference <= 1 month
│    ├─-- ComparisonLevel: Abs date difference <= 1 year
│    ├─-- ComparisonLevel: Abs date difference <= 10 year
│    ├─-- ComparisonLevel: All other comparisons
├─-- Comparison: Birth Place
│    ├─-- ComparisonLevel: Exact match on birth place with term frequency adjustments
│    ├─-- ComparisonLevel: All other
```

In [ ]:
import splink.comparison_library as cl
c = cl.DateOfBirthComparison(
            "dob", input_is_string=True
        ).get_comparison("duckdb")

for cl in c.as_dict()['comparison_levels'][1:]:
    print("")
    print(cl["sql_condition"])

In [11]:
import splink.comparison_library as cl

from splink import SettingsCreator

settings = SettingsCreator(
    link_type="dedupe_only",
    blocking_rules_to_generate_predictions=blocking_rules,
    comparisons=[
        cl.ForenameSurnameComparison(
            "first_name",
            "surname",
            forename_surname_concat_col_name="first_and_surname",
        ),
        cl.DateOfBirthComparison(
            "dob", input_is_string=True
        ),
        cl.PostcodeComparison("postcode_fake"),
        cl.ExactMatch("birth_place").configure(term_frequency_adjustments=True),
        cl.ExactMatch("occupation").configure(term_frequency_adjustments=True),
    ],
    retain_intermediate_calculation_columns=True,
    retain_matching_columns=True,
    additional_columns_to_retain=["cluster_ground_truth"]
)

In [12]:
from splink import Linker
linker = Linker(df, settings, db_api=db_api)

### Estimate the probability that two random records match

In [ ]:
linker.training.estimate_probability_two_random_records_match(
    [
        "l.first_name = r.first_name and l.surname = r.surname and l.dob = r.dob",
        "substr(l.first_name,1,2) = substr(r.first_name,1,2) and l.surname = r.surname and substr(l.postcode_fake,1,2) = substr(r.postcode_fake,1,2)",
        "l.dob = r.dob and l.postcode_fake = r.postcode_fake",
    ],
    recall=0.6,
)

### Estimate the `u` probabilities (direct estimation)

In [ ]:
linker.training.estimate_u_using_random_sampling(max_pairs=1e6)

### Estimate the `m` probabilities (unsupervised)

In [ ]:
training_blocking_rule = block_on("first_name", "surname")
training_session_names = (
    linker.training.estimate_parameters_using_expectation_maximisation(
        training_blocking_rule, estimate_without_term_frequencies=True
    )
)

In [ ]:
training_blocking_rule = block_on("dob")
training_session_dob = (
    linker.training.estimate_parameters_using_expectation_maximisation(
        training_blocking_rule, estimate_without_term_frequencies=True
    )
)

### Visualise the match weights

In [ ]:
linker.visualisations.match_weights_chart()

### Predict

In [ ]:
df_predict = linker.inference.predict(threshold_match_weight=-1)


sql = f"""
select *
from {df_predict.physical_name}
where

(unique_id_l = 'Q7964195-7' and unique_id_r = 'Q7964195-8')
or
(unique_id_l = 'Q2296770-1' and unique_id_r = 'Q2296770-6')

"""

rec = linker.misc.query_sql(sql)
rec

In [ ]:
input = rec.to_dict(orient='records')

linker.visualisations.waterfall_chart(input, filter_nulls=False)


### Cluster

In [ ]:
clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(df_predict, threshold_match_probability=0.9)
clusters.as_duckdbpyrelation().show(max_width=2000)

In [21]:
from IPython.display import IFrame

linker.visualisations.comparison_viewer_dashboard(
    df_predict,
    out_path="comparison_viewer.html",
    overwrite=True,
)


In [22]:
from IPython.display import IFrame

linker.visualisations.cluster_studio_dashboard(
    df_predict,
    clusters,
    "cluster_studio.html",
    sampling_method="by_cluster_size",
    overwrite=True,
)




## Evaluation

In [ ]:
linker.evaluation.unlinkables_chart()

In [ ]:
linker.evaluation.accuracy_analysis_from_labels_column("cluster_ground_truth", output_type="roc")

In [ ]:
linker.evaluation.accuracy_analysis_from_labels_column("cluster_ground_truth", output_type="threshold_selection")


https://www.robinlinacre.com/splink_example_charts/example_charts/interactive_clusters.html

https://www.robinlinacre.com/splink_example_charts/example_charts/splink3/labelling_tool_dedupe_only.html

https://www.robinlinacre.com/splink_example_charts/example_charts/interactive_clusters.html Kitty muggeridge 137438955949  Rothschild 889
